In [4]:
import yfinance as yf
import requests
import pandas as pd
import json
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

In [11]:
import requests
from bs4 import BeautifulSoup

# URL for Apple Key Statistics on Yahoo Finance
url = "https://finance.yahoo.com/quote/AAPL/key-statistics/"

# Headers to mimic a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
}

# Sending a GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extracting specific statistics
    statistics = {}
    for row in soup.select("table tbody tr"):
        # Extracting title and value
        columns = row.find_all("td")
        if len(columns) == 2:
            key = columns[0].get_text(strip=True)
            value = columns[1].get_text(strip=True)
            statistics[key] = value

    # Printing the statistics
    for key, value in statistics.items():
        print(f"{key}: {value}")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")


Fiscal Year Ends: 9/28/2024
Most Recent Quarter  (mrq): 9/28/2024
Profit Margin: 23.97%
Operating Margin  (ttm): 31.17%
Return on Assets  (ttm): 21.46%
Return on Equity  (ttm): 157.41%
Revenue  (ttm): 391.03B
Revenue Per Share  (ttm): 25.49
Quarterly Revenue Growth  (yoy): 6.10%
Gross Profit  (ttm): --
EBITDA: 134.66B
Net Income Avi to Common  (ttm): 93.74B
Diluted EPS  (ttm): 6.09
Quarterly Earnings Growth  (yoy): -35.80%
Total Cash  (mrq): 65.17B
Total Cash Per Share  (mrq): 4.31
Total Debt  (mrq): 119.06B
Total Debt/Equity  (mrq): 209.06%
Current Ratio  (mrq): 0.87
Book Value Per Share  (mrq): 3.77
Operating Cash Flow  (ttm): 118.25B
Levered Free Cash Flow  (ttm): 110.85B
Beta (5Y Monthly): 1.24
52 Week Range3: 24.42%
S&P 500 52-Week Change3: 28.91%
52 Week High3: 250.80
52 Week Low3: 164.08
50-Day Moving Average3: 231.47
200-Day Moving Average3: 208.28
Avg Vol (3 month)3: 47.77M
Avg Vol (10 day)3: 39.7M
Shares Outstanding5: 15.12B
Implied Shares Outstanding6: 15.33B
Float8: 15.09B


In [16]:
import json
a = []
with open("symbol_list.json", "r") as f:
    df = json.load(f)
a = [i['ticker'] for j, i in df.items()]
len(a)

10016

10016